# Genotype data (SNP) QC pipeline

SNP 데이터를 preprocessing하는 과정의 pipeline입니다.


필요한 소프트웨어는 PLINK1.9버전(https://www.cog-genomics.org/plink/) 입니다.

폴더 안에는 PLINK와 preprocess할 genotype data가 binary plink format(bed,bim,fam)의 확장자를 가진 같은 이름으로 존재해야 합니다.


Phenotype은 넣지 않아도 돌아가게 만들었습니다.

In [1]:
'''
이 곳의 파라미터를 바꿈으로써 preprocessing의 여러 threshold를 바꿀 수 있습니다.
최종결과는 'ABCD_finished.(fbed,bim,fam)'파일로 나오게 됩니다.
'''
#작업할 파일 공간(이 안에 genotype data와 plink 소프트웨어를 넣어두어야 합니다)
Filepath = '/home/hjd/Desktop/PLINK/test'

#Genotype data의 파일명입니다.
Filename = 'ABCD_release_3.0_QCed'

#제외해야 할 목록이 있다면 그 목록이 적힌 텍스트파일의 이름을 쓰면 됩니다. 물론 이 파일도 filepath 안에 넣어놔야 합니다.
Bad_batch_file = 'batch_bad.txt'

#Phenotype의 정보가 들어있는 파일이름입니다. 없으면 놔두셔도 무방합니다.
#파일의 첫column은 Family ID, 두번째column은 Individual ID, 그리고 3번째 column이 phenotype(trait)입니다.
Phenotype_file = None #'qtrait.txt'

#--geno,--mind,MAF,HWE의 threshold를 설정합니다.
Genotype_miss = None
Individual_miss = None
MAF = None
HWE = None

#몇번부터 몇번 염색체까지 쓸 것인지 설정합니다.
Chromosome = None #[,]

#Prune에 필요한 수치를 설정합니다.
Prune_window_size = None
Prune_sliding = None
Prune_r2 = None

#Heterozygosity에서 몇 standard deviation이상을 제외할건지 threshold를 설정합니다.
Heterozygosity_SD = 4

#Relatedness 분석에서 최솟값과 최댓값을 설정합니다.
Relatedness_min = 0.4
Relatedness_max = None

In [2]:
#기본세팅입니다. 값을 입력하지 않으면 이 값을 바탕으로 진행됩니다.

config = {
    'Filepath' : None,
    'Filename' : None,
    'Bad_batch_file' : None,
    'Phenotype_file' : None,
    'Genotype_miss' : 0.05,
    'Individual_miss' : 0.05,
    'Minor allele frequency' : 0.01,
    'Chromosome' : [1,22],
    'Hardy weinberg equil' : 1e-6,
    'Prune' : {
        'window_size': 200,
        'sliding': 50,
        'r2': 0.25,
    },
    'Heterozygosity(SD)' : 3,
    'Relatedness' : {
        'min' : None,
        'max' : None,
    },
}

In [4]:
import os, sys
import numpy as np
import pandas as pd

#Config
config['Filepath'] = Filepath
config['Filename'] = Filename
config['Bad_batch_file'] = Bad_batch_file
config['Phenotype_file'] = Phenotype_file
config['Genotype_miss'] = config['Genotype_miss'] if Genotype_miss == None else Genotype_miss
config['Individual_miss'] = config['Individual_miss'] if Individual_miss == None else Individual_miss
config['Minor allele frequency'] = config['Minor allele frequency'] if MAF == None else MAF
config['Chromosome'] = config['Chromosome'] if Chromosome == None else Chromosome
config['Prune']['window_size'] = config['Prune']['window_size'] if Prune_window_size == None else Prune_window_size
config['Prune']['sliding'] = config['Prune']['sliding'] if Prune_sliding == None else Prune_sliding
config['Prune']['r2'] = config['Prune']['r2'] if Prune_r2 == None else Prune_r2
config['Heterozygosity(SD)'] = config['Heterozygosity(SD)'] if Heterozygosity_SD == None else Heterozygosity_SD
config['Relatedness']['min'] = Relatedness_min
config['Relatedness']['max'] = Relatedness_max

os.chdir(config['Filepath'])
os.mkdir("Final_results")
#Save config
import json
with open('Final_results/Used_parameters.json', 'w') as f:
        json.dump(config, f,indent = 0)

print('=================START Preprocessing!======================\n')
print('=================1.Bad batch removing======================')
#Exclude bad batch
if config['Bad_batch_file'] != None:
    print('Bad batch file detected! Processing...')
    os.mkdir('1.Bad_batch_removed')
    os.system('./plink --bfile ABCD_release_3.0_QCed --remove {} --out 1.Bad_batch_removed/ABCD_excluded --make-bed'.format(config['Bad_batch_file']))
else:
    print('No bad batch file detected. Anyway, New file will be made!')
    os.mkdir('1.Bad_batch_removed')
    os.system('./plink --bfile ABCD_release_3.0_QCed --out 1.Bad_batch_removed/ABCD_excluded --make-bed'.format(config['Bad_batch_file']))

print('=================1.Bad batch removed======================\n')

print('=================1.Checking phenotype======================')
if config['Phenotype_file'] != None:
    if os.path.isfile(config['Phenotype_file']):
        os.mkdir('Phenotyped')
        print('Phenotype file detected!! Processing...')
        os.system('./plink --bfile 1.Bad_batch_removed/ABCD_excluded --pheno {} --out Phenotyped/ABCD_phenotyped --make-bed'.format(config['Phenotype_file']))
    else:
        print('No phenotype file detected. Run preprocess without phenotype.')
        pass
else: print('No phenotype file detected. Run preprocess without phenotype.')
print('===================Step 1 completed=======================\n')

#Exclude missing genotype
print('======2.Start excluding missing snps(--geno), threshold = {}======'.format(config['Genotype_miss']))
os.mkdir('2.Missing_genotype')
if config['Phenotype_file'] != None:
    os.system('./plink --bfile Phenotyped/ABCD_phenotyped --geno {} --out 2.Missing_genotype/ABCD_geno --make-bed'.format(config['Genotype_miss']))
else:
    os.system('./plink --bfile 1.Bad_batch_removed/ABCD_excluded --geno {} --out 2.Missing_genotype/ABCD_geno --make-bed'.format(config['Genotype_miss']))
print('===================Step 2 completed=======================\n')
    
print('======3.Start excluding missing individuals(--mind), threshold = {}====='.format(config['Individual_miss']))
#Exclude missing individuals
os.mkdir('3.Missing_individuals')
os.system('./plink --bfile 2.Missing_genotype/ABCD_geno --mind {} --out 3.Missing_individuals/ABCD_mind --make-bed'.format(config['Individual_miss']))
print('===================Step 3 completed=======================\n')

#Impute sex
print('=================4.Imputing sex======================')
os.mkdir('4.Impute_sex')
os.system('./plink --bfile 3.Missing_individuals/ABCD_mind --impute-sex --out 4.Impute_sex/ABCD_sex --make-bed')
print('===================Step 4 completed=======================\n')

#Choose which chromosome to use
print('======5.Choosing chromosome : {} to {}======'.format(config['Chromosome'][0],
                                                         config['Chromosome'][1]))
os.mkdir('5.Choose_chromosome')
os.system('./plink --bfile 4.Impute_sex/ABCD_sex --chr {}-{} --out 5.Choose_chromosome/ABCD_chr --make-bed'.format(config['Chromosome'][0],
                                                                                                                  config['Chromosome'][1]))
print('===================Step 5 completed=======================\n')

#Minor Allele Frequency
print('======6.Minor allele frequency, threshold = {}======'.format(config['Minor allele frequency']))
os.mkdir('6.Minor_allele_frequency')
os.system('./plink --bfile 5.Choose_chromosome/ABCD_chr --maf {} --out 6.Minor_allele_frequency/ABCD_maf --make-bed'.format(config['Minor allele frequency']))
print('===================Step 6 completed=======================\n')

#Hardy Weinberg Equilibrium
print('======7.Hardy weinberg equilibrium, threshold = {}======'.format(config['Hardy weinberg equil']))
os.mkdir('7.HWE')
os.system('./plink --bfile 6.Minor_allele_frequency/ABCD_maf --hwe {} --out 7.HWE/ABCD_hwe --make-bed'.format(config['Hardy weinberg equil']))
print('===================Step 7 completed=======================\n')

#Pruning
print('====================8.Start pruning... ===================')
print('window size = {}'.format(config['Prune']['window_size']))
print('sliding = {}'.format(config['Prune']['sliding']))
print('r2 = {}'.format(config['Prune']['r2']))
os.mkdir('8.Pruning')
os.system('./plink --bfile 7.HWE/ABCD_hwe --indep-pairwise {} {} {} --out 8.Pruning/ABCD_pruned'.format(config['Prune']['window_size'],
                                                                                       config['Prune']['sliding'],
                                                                                       config['Prune']['r2']))

#Make pruned dataset
print('======Making pruned data... ======')
os.system('./plink --bfile 7.HWE/ABCD_hwe --extract 8.Pruning/ABCD_pruned.prune.in --out 8.Pruning/ABCD_pruned --make-bed')
print('===================Step 8 completed=======================\n')

print('===========9.Start heterozygosity calculation ============')
#Heterozygosity with pruned dataset
os.mkdir('9.Heterozygosity')
os.system('./plink --bfile 8.Pruning/ABCD_pruned --het --out 9.Heterozygosity/ABCD_het')

#Make removal list with heterozygosity calculation
ABCD_het = pd.read_csv('9.Heterozygosity/ABCD_het.het', sep = ' ', low_memory = False,
                       skipinitialspace=True)

#Calculating heterozygosity
ABCD_het['het'] = (ABCD_het['N(NM)']-ABCD_het['O(HOM)'])/ABCD_het['N(NM)']
mean = np.mean(ABCD_het['het'])
std = np.std(ABCD_het['het'])
print('======Exclude {}SD away from mean value ======'.format(config['Heterozygosity(SD)']))
over_out = ABCD_het.loc[ABCD_het['het'] > mean+int('{}'.format(config['Heterozygosity(SD)']))*std,:]
under_out = ABCD_het.loc[ABCD_het['het'] < mean-int('{}'.format(config['Heterozygosity(SD)']))*std, :]
remove = pd.concat([over_out,under_out], axis=0)
remove.to_csv('het_remove.txt', sep = ' ', header=None, index = False)
print('======het_remove file has been made!======')

os.system('./plink --bfile 7.HWE/ABCD_hwe --remove het_remove.txt --out 9.Heterozygosity/ABCD_het_removed --make-bed;')
print('===================Step 9 completed=======================\n')

print('======10.Calculating Relatedness...this takes time, be patient :) ======')
print('====== min : {}, max : {}======'.format(config['Relatedness']['min'],config['Relatedness']['max']))
os.mkdir('10.Relatedness')
if config['Relatedness']['min'] != None and config['Relatedness']['max'] == None:
    os.system('./plink --bfile 8.Pruning/ABCD_pruned --genome --min {} --out 10.Relatedness/ABCD_genome'.format(config['Relatedness']['min']))
if config['Relatedness']['min'] == None and config['Relatedness']['max'] != None:
    os.system('./plink --bfile 8.Pruning/ABCD_pruned --genome --max {} --out 10.Relatedness/ABCD_genome'.format(config['Relatedness']['max']))
if config['Relatedness']['min'] != None and config['Relatedness']['max'] != None:
    os.system('./plink --bfile 8.Pruning/ABCD_pruned --genome --min {} --max {} --out 10.Relatedness/ABCD_genome'.format(config['Relatedness']['min'],
                                                                                                         config['Relatedness']['max']))
if config['Relatedness']['min'] == None and config['Relatedness']['max'] == None:
    os.system('./plink --bfile 8.Pruning/ABCD_pruned --genome --out 10.Relatedness/ABCD_genome')

ABCD_genome = pd.read_csv('10.Relatedness/ABCD_genome.genome', sep = ' ', low_memory = False,
                       skipinitialspace=True)

#get missing rate information
os.system('./plink --bfile 1.Bad_batch_removed/ABCD_excluded --missing --out 10.Relatedness/ABCD_miss')
miss = pd.read_csv('10.Relatedness/ABCD_miss.imiss', sep = ' ', low_memory = False,
                       skipinitialspace=True)

print('========Comparing individuals========')
#Compare individuals missing rate and leave only good one for each pairs
low_call_list = []
low_call_val = []
for i in range(ABCD_genome.shape[0]):
    f1 = ABCD_genome.loc[i,'FID1']
    p1 = ABCD_genome.loc[i,'IID1']
    f2 = ABCD_genome.loc[i,'FID2']
    p2 = ABCD_genome.loc[i,'IID2']
    c1 = float(miss.loc[miss['IID'] == p1, 'F_MISS'])
    c2 = float(miss.loc[miss['IID'] == p2, 'F_MISS'])
    if c1>c2:
        low_call_list.append([f1,p1])
        low_call_val.append([c1,c2])
    elif c2>c1:
        low_call_list.append([f2,p2])
        low_call_val.append([c1,c2])
    else: continue
        
#Save low call individual list
print('========Saving low call individual text file========\n')
f = open("low_call.txt", 'w')
for i in range(len(low_call_list)):
    a1 = str(low_call_list[i]).replace("['",'')
    a2 = a1.replace("']",'')
    a3 = a2.replace("', '",' ')
    data = a3 + '\n'
    f.write(data)
f.close()
print('===================Step 10 completed=======================\n')
#Exclude low call list from data
os.system('./plink --bfile 9.Heterozygosity/ABCD_het_removed --remove low_call.txt --out Final_results/ABCD_QC_finished --make-bed')
print("=============Genotype preprocessing done!!!==================")

=================START Preprocessing!======================

=================1.Bad batch removing======================
Bad batch file detected! Processing...
=================1.Bad batch removed======================

=================1.Checking phenotype======================
No phenotype file detected. Run preprocess without phenotype.
===================Step 1 completed=======================

======2.Start excluding missing snps(--geno), threshold = 0.05======
===================Step 2 completed=======================

======3.Start excluding missing individuals(--mind), threshold = 0.05=====
===================Step 3 completed=======================

=================4.Imputing sex======================
===================Step 4 completed=======================

======5.Choosing chromosome : 1 to 22======
===================Step 5 completed=======================

======6.Minor allele frequency, threshold = 0.01======
===================Step 6 completed=======================

=